In [1]:
import pandas as pd

In [2]:
#dataset indios PIMA - diabetes
df = pd.read_csv('datasets/diabetes.csv')

In [3]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [4]:
df['Outcome'].value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

In [5]:
#proporção dos que tem diabetes no dataset
268/len(df) * 100

34.89583333333333

#### Se chutar 65% das vezes acerta que não tem diabetes

In [8]:
#criando matriz de features e vetor alvo
X = df.drop('Outcome', axis=1)
y = df[['Outcome']]

In [9]:
X.shape, y.shape

((768, 8), (768, 1))

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=df['Outcome'])

In [12]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((614, 8), (614, 1), (154, 8), (154, 1))

In [13]:
y_train.value_counts()

Outcome
0          400
1          214
Name: count, dtype: int64

In [14]:
!pip install imblearn

In [15]:
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

In [16]:
def my_cross_validation(model, X, y, oversampling=False):
    kfold = KFold(n_splits=10) #cv=10
    accs_split = []
    for idx, (idx_train, idx_validation) in enumerate(kfold.split(X)):
        X_split_train = X.iloc[idx_train,:]
        y_split_train = y.iloc[idx_train,:]
        # se oversampling, aplicar só no split de treino, pois o de validação precisa manter a característica de balance original
        if oversampling:
            sm = SMOTE(random_state=42)
            X_split_train, y_split_train = sm.fit_resample(X_split_train, y_split_train)
        
        model.fit(X_split_train, y_split_train.values.flatten())
        
        X_split_validation = X.iloc[idx_validation, :]
        y_split_validation = y.iloc[idx_validation, :]
        
        y_pred_validation = model.predict(X_split_validation)
        acc_split = accuracy_score(y_split_validation, y_pred_validation)
        accs_split.append(acc_split)
        
        print(f'accuracy at split {idx}: {acc_split}')
    return accs_split

#### Vamos testar um classificador em comitê (emsemble), mas pode ser testado com qualquer classificador

In [19]:
from sklearn.ensemble import HistGradientBoostingClassifier
from statistics import mean

model_hgb = HistGradientBoostingClassifier()

In [22]:
avg_acc_without_smote = mean(my_cross_validation(model_hgb, X_train, y_train, oversampling=False))*100
avg_acc_with_smote = mean(my_cross_validation(model_hgb, X_train, y_train, oversampling=True))*100

print(f'avg_acc_without_smote {avg_acc_without_smote:.02f}, avg_acc_with_smote: {avg_acc_with_smote:.02f}')

accuracy at split 0: 0.8548387096774194
accuracy at split 1: 0.7258064516129032
accuracy at split 2: 0.7741935483870968
accuracy at split 3: 0.7419354838709677
accuracy at split 4: 0.7377049180327869
accuracy at split 5: 0.8032786885245902
accuracy at split 6: 0.6885245901639344
accuracy at split 7: 0.7213114754098361
accuracy at split 8: 0.7213114754098361
accuracy at split 9: 0.6721311475409836
accuracy at split 0: 0.8387096774193549
accuracy at split 1: 0.7096774193548387
accuracy at split 2: 0.7096774193548387
accuracy at split 3: 0.7419354838709677
accuracy at split 4: 0.7213114754098361
accuracy at split 5: 0.7704918032786885
accuracy at split 6: 0.6721311475409836
accuracy at split 7: 0.7377049180327869
accuracy at split 8: 0.7377049180327869
accuracy at split 9: 0.7049180327868853
avg_acc_without_smote 74.41, avg_acc_with_smote: 73.44


#### Mas qual seria a forma errada de fazer? o SMOTE antes de passar para a cross validation que irá separar treino e validação na base já 'corrigida'

In [ ]:
from sklearn.model_selection import cross_val_score
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)
mean(cross_val_score(model_hgb, X_train, y_train.values.flatten()))*100 #ESTA MÉDIA DE ACURÁCIA NÃO É REAL, FOI ENVIESADA PELO DEV

81.75